In [1]:
import pandas as pd
import numpy as np
import pathlib
from pdfminer.high_level import extract_text
from PyPDF2 import PdfFileMerger
from datetime import datetime

/home/dastudillo/.local/lib/python3.6/site-packages/pdfminer/pdfdocument.py:22: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend


# Load data

In [2]:
#Assign the path where your interrogation reports are located
p = pathlib.Path('/home/dastudillo/PDMS/programming_reports/PR04/')   #insert your directory here
pdf_files = [x.stem for x in p.glob('*.pdf')] #extract name of files in pdf format only

In [3]:
#create df with file names in column
pdf_df = pd.DataFrame(pdf_files, columns = ['filename'])

In [4]:
#You are only extracting the file names
#so you have re add the .pdf sufix in order to process files using the PDF library tools
pdf_df['filename'] = pdf_df['filename'].astype(str) + '.pdf'
pdf_list = list(pdf_df['filename'])
#pdf_list

In [5]:
#Create list of paths for each interrogation report you want to analyze together
pdf_paths = ['/home/dastudillo/PDMS/programming_reports/PR04/' + x for x in pdf_list]
print('Parsing ' + str(len(pdf_paths)) + ' report(s)')

Parsing 1 report(s)


# Parse single or multiple report(s)

In [6]:
list_lines = []
for i in range(len(pdf_paths)):
    text = extract_text(pdf_paths[i])
    lines = text.split('\n')
    lines = [value for value in lines if value != '']
    list_lines.append(lines)
    i+=1

In [7]:
final_list_lines = []
for i in range(len(list_lines)):
    temp = [e for e in list_lines[i] if e not in ('Impedance Alert',
                                         'Impedance is unmeasured for electrodes LSGC2, LSGC3, LSGC4, RSGC1, RSGC2, RSGC3, and RSGC4.',
                                         'Device Alerts')]
    final_list_lines.append(temp)
    i+=1

In [8]:
#Detection settings
start_time = []
det_status = []
capture_win = []
inversion_logic = []
pattern_a = []
pattern_b = []
first_detector = []
second_detector = []

for i in range(len(final_list_lines)):
    start_time.append(final_list_lines[i][final_list_lines[i].index('Report Type')-1])
    det_status.append(final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1])
    capture_win.append(final_list_lines[i][final_list_lines[i].index('Length of Stored ECoG')+1])
    pattern_a.append(final_list_lines[i][final_list_lines[i].index('Pattern A')+6])
    pattern_b.append(final_list_lines[i][final_list_lines[i].index('Pattern B')+6])
    first_detector.append(final_list_lines[i][final_list_lines[i].index('Pattern A')+4])
    second_detector.append(final_list_lines[i][final_list_lines[i].index('Pattern B')+4])
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Enabled':
        inversion_logic.append('Not Inverted')
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Disabled':
        inversion_logic.append('null')        
    i+=1

In [9]:
#A1 detector

A1_status = []
A1_min_freq_win = []
A1_min_freq_count = [] 
A1_hysteresis = []
A1_min_amp = []
A1_min_width = []
A1_det_analysis_win = [] 
A1_band_thresh = []
A1_persistance = []
A1_min_freq = [] 
A1_max_freq = []
A1_min_amp_cli = [] 
A1_min_duration = []

 
for i in range(len(final_list_lines)):
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Enabled' and 'Pattern A, First Detector (Not Inverted)' in final_list_lines[i]:
        A1_status.append('On')
        A1_min_freq_win.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-8])
        A1_min_freq_count.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-7])
        A1_hysteresis.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-6])
        A1_min_amp.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-5])
        A1_min_width.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-4])
        A1_det_analysis_win.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-3])
        A1_band_thresh.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-2])
        A1_persistance.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')-1])
        A1_min_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')+6])
        A1_max_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')+7])
        A1_min_amp_cli.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')+8])
        A1_min_duration.append(final_list_lines[i][final_list_lines[i].index('Pattern A, First Detector (Not Inverted)')+9])
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Disabled' or 'Pattern A, First Detector (Not Inverted)' not in final_list_lines[i]:
        A1_status.append('Off')
        A1_min_freq_win.append('null')
        A1_min_freq_count.append('null')
        A1_hysteresis.append('null')
        A1_min_amp.append('null')
        A1_min_width.append('null')
        A1_det_analysis_win.append('null')
        A1_band_thresh.append('null')
        A1_persistance.append('null')
        A1_min_freq.append('null')
        A1_max_freq.append('null')
        A1_min_amp_cli.append('null')
        A1_min_duration.append('null')
        i+=1       

In [10]:
#A2 detector

A2_status = []
A2_min_freq_win = []
A2_min_freq_count = [] 
A2_hysteresis = []
A2_min_amp = []
A2_min_width = []
A2_det_analysis_win = [] 
A2_band_thresh = []
A2_persistance = []
A2_min_freq = [] 
A2_max_freq = []
A2_min_amp_cli = [] 
A2_min_duration = []

 
for i in range(len(final_list_lines)):
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Enabled' and 'Pattern A, Second Detector (Not Inverted)' in final_list_lines[i]:
        A2_status.append('On')
        A2_min_freq_win.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+15])
        A2_min_freq_count.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+16])
        A2_hysteresis.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+17])
        A2_min_amp.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+18])
        A2_min_width.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+19])
        A2_det_analysis_win.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+20])
        A2_band_thresh.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+21])
        A2_persistance.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+22])
        A2_min_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+27])
        A2_max_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+28])
        A2_min_amp_cli.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+29])
        A2_min_duration.append(final_list_lines[i][final_list_lines[i].index('Pattern A, Second Detector (Not Inverted)')+30])
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Disabled' or 'Pattern A, Second Detector (Not Inverted)' not in final_list_lines[i]:
        A2_status.append('Off')
        A2_min_freq_win.append('null')
        A2_min_freq_count.append('null')
        A2_hysteresis.append('null')
        A2_min_amp.append('null')
        A2_min_width.append('null')
        A2_det_analysis_win.append('null')
        A2_band_thresh.append('null')
        A2_persistance.append('null')
        A2_min_freq.append('null')
        A2_max_freq.append('null')
        A2_min_amp_cli.append('null')
        A2_min_duration.append('null')
        i+=1   

In [11]:
#B1 detector

B1_status = []
B1_min_freq_win = []
B1_min_freq_count = [] 
B1_hysteresis = []
B1_min_amp = []
B1_min_width = []
B1_det_analysis_win = [] 
B1_band_thresh = []
B1_persistance = []
B1_min_freq = [] 
B1_max_freq = []
B1_min_amp_cli = [] 
B1_min_duration = []

for i in range(len(final_list_lines)):
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Enabled' and 'Pattern B, First Detector (Not Inverted)' in final_list_lines[i]:
        B1_status.append('On')
        B1_min_freq_win.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+15])
        B1_min_freq_count.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+16])
        B1_hysteresis.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+17])
        B1_min_amp.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+18])
        B1_min_width.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+19])
        B1_det_analysis_win.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+20])
        B1_band_thresh.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+21])
        B1_persistance.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+22])
        B1_min_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+27])
        B1_max_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+28])
        B1_min_amp_cli.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+29])
        B1_min_duration.append(final_list_lines[i][final_list_lines[i].index('Pattern B, First Detector (Not Inverted)')+30])
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Disabled' or 'Pattern B, First Detector (Not Inverted)' not in final_list_lines[i]:
        B1_status.append('Off')
        B1_min_freq_win.append('null')
        B1_min_freq_count.append('null')
        B1_hysteresis.append('null')
        B1_min_amp.append('null')
        B1_min_width.append('null')
        B1_det_analysis_win.append('null')
        B1_band_thresh.append('null')
        B1_persistance.append('null')
        B1_min_freq.append('null')
        B1_max_freq.append('null')
        B1_min_amp_cli.append('null')
        B1_min_duration.append('null')
        i+=1 

In [12]:
#B2 detector

B2_status = []
B2_min_freq_win = []
B2_min_freq_count = [] 
B2_hysteresis = []
B2_min_amp = []
B2_min_width = []
B2_det_analysis_win = [] 
B2_band_thresh = []
B2_persistance = []
B2_min_freq = [] 
B2_max_freq = []
B2_min_amp_cli = [] 
B2_min_duration = []

 
for i in range(len(final_list_lines)):
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Enabled' and 'Pattern B, Second Detector (Not Inverted)' in final_list_lines[i]:
        B2_status.append('On')
        B2_min_freq_win.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+15])
        B2_min_freq_count.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+16])
        B2_hysteresis.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+17])
        B2_min_amp.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+18])
        B2_min_width.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+19])
        B2_det_analysis_win.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+20])
        B2_band_thresh.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+21])
        B2_persistance.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+22])
        B2_min_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+27])
        B2_max_freq.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+28])
        B2_min_amp_cli.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+29])
        B2_min_duration.append(final_list_lines[i][final_list_lines[i].index('Pattern B, Second Detector (Not Inverted)')+30])
    if final_list_lines[i][final_list_lines[i].index('Programming Set Name')-1]=='Disabled' or 'Pattern B, Second Detector (Not Inverted)' not in final_list_lines[i]:
        B2_status.append('Off')
        B2_min_freq_win.append('null')
        B2_min_freq_count.append('null')
        B2_hysteresis.append('null')
        B2_min_amp.append('null')
        B2_min_width.append('null')
        B2_det_analysis_win.append('null')
        B2_band_thresh.append('null')
        B2_persistance.append('null')
        B2_min_freq.append('null')
        B2_max_freq.append('null')
        B2_min_amp_cli.append('null')
        B2_min_duration.append('null')
        i+=1 

In [13]:
#Stim settings
con1 = [] 
con2 = [] 
con3 = [] 
con4 = [] 
con5 = [] 
con6 = [] 
con7 = [] 
con8 = [] 
con9 = []
amp = []
freq = []
PW = []
burst_dur = []
charge_density = []
PEI =[]
tx_status = []
cap = []
cap_reset = []
 
for i in range(len(final_list_lines)):
    if 'Therapy #4' in (final_list_lines[i]):
        con1.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+9] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+18])
        con2.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+10] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+19])
        con3.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+11] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+20])
        con4.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+12] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+21])
        con5.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+13] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+22])
        con6.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+14] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+23])
        con7.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+15] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+24])
        con8.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+16] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+25])
        con9.append(final_list_lines[i][final_list_lines[i].index('Therapy #4')+17] + ' ' + final_list_lines[i][final_list_lines[i].index('Therapy #4')+26])
        amp.append(final_list_lines[i][final_list_lines[i].index('Charge Density')-5])
        freq.append(final_list_lines[i][final_list_lines[i].index('Charge Density')-4])
        PW.append(final_list_lines[i][final_list_lines[i].index('Charge Density')-3])
        burst_dur.append(final_list_lines[i][final_list_lines[i].index('Charge Density')-2])
        charge_density.append(final_list_lines[i][final_list_lines[i].index('Charge Density')+1])
        PEI.append(final_list_lines[i][final_list_lines[i].index('Post-Episode Interval')+1])
        tx_status.append(final_list_lines[i][final_list_lines[i].index('Withholding')+1])
        cap.append(final_list_lines[i][final_list_lines[i].index('Withholding')+3])
        cap_reset.append(final_list_lines[i][final_list_lines[i].index('Withholding')+4])
    if ('Therapy #4' not in (final_list_lines[i]) and 'Therapy #1' in (final_list_lines[i])):
        con1.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+1] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+1])
        con2.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+2] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+2])
        con3.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+3] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+3])
        con4.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+4] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+4])
        con5.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+5] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+5])
        con6.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+6] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+6])
        con7.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+7] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+7])
        con8.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+8] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+8])
        con9.append(final_list_lines[i][final_list_lines[i].index('Therapy #1')+9] + ' ' + final_list_lines[i][final_list_lines[i].index('Synch Slope')+9])
        amp.append(final_list_lines[i][final_list_lines[i].index('Synch Slope')+10])
        freq.append(final_list_lines[i][final_list_lines[i].index('Synch Slope')+11])
        PW.append(final_list_lines[i][final_list_lines[i].index('Synch Slope')+12])
        burst_dur.append(final_list_lines[i][final_list_lines[i].index('Synch Slope')+13])
        charge_density.append(final_list_lines[i][final_list_lines[i].index('Synch Slope')+14])
        PEI.append(final_list_lines[i][final_list_lines[i].index('Post-Episode Interval')+1])
        tx_status.append(final_list_lines[i][final_list_lines[i].index('Withholding')+1])
        cap.append(final_list_lines[i][final_list_lines[i].index('Withholding')+3])
        cap_reset.append(final_list_lines[i][final_list_lines[i].index('Withholding')+4])
    if 'Therapy #1' not in (final_list_lines[i]):
        con1.append('null')
        con2.append('null')
        con3.append('null')
        con4.append('null')
        con5.append('null')
        con6.append('null')
        con7.append('null')
        con8.append('null')
        con9.append('null')
        amp.append('null')
        freq.append('null')
        PW.append('null')
        burst_dur.append('null')
        charge_density.append('null')
        PEI.append('null')
        tx_status.append('Disabled')
        cap.append('null')
        cap_reset.append('null')
        i+=1

# Tabulate data

In [14]:
#Create list of lists that will be use as input data for final dataframe columns
final_list = []
for i in range(len(start_time)):
    final_list.append([start_time[i], cap[i], cap_reset[i], capture_win[i], det_status[i], tx_status[i],
                       inversion_logic[i], pattern_a[i], pattern_b[i], A1_status[i], first_detector[i],
                       A1_min_freq[i], A1_max_freq[i], A1_min_amp_cli[i], A1_det_analysis_win[i],
                       A1_band_thresh[i], A1_persistance[i], A1_min_freq_win[i], A1_min_freq_count[i],
                       A1_hysteresis[i], A1_min_amp[i], A1_min_width[i], A1_min_duration[i],
                       B1_status[i], first_detector[i], B1_min_freq[i], B1_max_freq[i], B1_min_amp_cli[i],
                       B1_det_analysis_win[i], B1_band_thresh[i], B1_persistance[i], B1_min_freq_win[i],
                       B1_min_freq_count[i], B1_hysteresis[i], B1_min_amp[i], B1_min_width[i],
                       B1_min_duration[i], A2_status[i], second_detector[i], A2_min_freq[i], A2_max_freq[i],
                       A2_min_amp_cli[i], A2_det_analysis_win[i], A2_band_thresh[i], A2_persistance[i],
                       A2_min_freq_win[i], A2_min_freq_count[i], A2_hysteresis[i], A2_min_amp[i],
                       A2_min_width[i], A2_min_duration[i], B2_status[i], second_detector[i], B2_min_freq[i],
                       B2_max_freq[i], B2_min_amp_cli[i], B2_det_analysis_win[i], B2_band_thresh[i],
                       B2_persistance[i], B2_min_freq_win[i], B2_min_freq_count[i], B2_hysteresis[i],
                       B2_min_amp[i], B2_min_width[i], B2_min_duration[i], con1[i], con2[i], con3[i],
                       con4[i], con5[i], con6[i], con7[i], con8[i], con9[i], amp[i], freq[i], PW[i],
                       burst_dur[i], charge_density[i], PEI[i]])
    i+=1

In [15]:
#Create initial dataframe
to_df = pd.DataFrame(final_list, columns = ['StartTime', 
                                            'TherapyLimitPerDay',
                                            'TherapyLimitResetTime',
                                            'CaptureWindow',
                                            'DetectionStatus',
                                            'StimulationStatus',
                                            'InversionLogic',
                                            'PatternALogic',
                                            'PatternBLogic',
                                            'DetectorA1_BandpassStatus',
                                            'DetectorA1_Channels',
                                            'DetectorA1_MinFreq',
                                            'DetectorA1_MaxFreq',
                                            'DetectorA1_MinAmp_ClinMode',
                                            'DetectorA1_DetectAnalysisWindowSize',
                                            'DetectorA1_BandpassThreshold',
                                            'DetectorA1_DetectionPersistence',
                                            'DetectorA1_MinFreqWindow',
                                            'DetectorA1_MinFreqCountCriteria',
                                            'DetectorA1_Hysteresis',
                                            'DetectorA1_MinAmp_TechMode',
                                            'DetectorA1_MinWidth',
                                            'DetectorA1_MinDuration',
                                            'DetectorB1_BandpassStatus',
                                            'DetectorB1_Channels',
                                            'DetectorB1_MinFreq',
                                            'DetectorB1_MaxFreq',
                                            'DetectorB1_MinAmp_ClinMode',
                                            'DetectorB1_DetectAnalysisWindowSize',
                                            'DetectorB1_BandpassThreshold',
                                            'DetectorB1_DetectionPersistence',
                                            'DetectorB1_MinFreqWindow',
                                            'DetectorB1_MinFreqCountCriteria',
                                            'DetectorB1_Hysteresis',
                                            'DetectorB1_MinAmp_TechMode',
                                            'DetectorB1_MinWidth',
                                            'DetectorB1_MinDuration',
                                            'DetectorA2_BandpassStatus',
                                            'DetectorA2_Channels',
                                            'DetectorA2_MinFreq',
                                            'DetectorA2_MaxFreq',
                                            'DetectorA2_MinAmp_ClinMode',
                                            'DetectorA2_DetectAnalysisWindowSize',
                                            'DetectorA2_BandpassThreshold',
                                            'DetectorA2_DetectionPersistence',
                                            'DetectorA2_MinFreqWindow',
                                            'DetectorA2_MinFreqCountCriteria',
                                            'DetectorA2_Hysteresis',
                                            'DetectorA2_MinAmp_TechMode',
                                            'DetectorA2_MinWidth',
                                            'DetectorA2_MinDuration',
                                            'DetectorB2_BandpassStatus',
                                            'DetectorB2_Channels',
                                            'DetectorB2_MinFreq',
                                            'DetectorB2_MaxFreq',
                                            'DetectorB2_MinAmp_ClinMode',
                                            'DetectorB2_DetectAnalysisWindowSize',
                                            'DetectorB2_BandpassThreshold',
                                            'DetectorB2_DetectionPersistence',
                                            'DetectorB2_MinFreqWindow',
                                            'DetectorB2_MinFreqCountCriteria',
                                            'DetectorB2_Hysteresis',
                                            'DetectorB2_MinAmp_TechMode',
                                            'DetectorB2_MinWidth',
                                            'DetectorB2_MinDuration',
                                            'contact_1',
                                            'contact_2',
                                            'contact_3',
                                            'contact_4',
                                            'contact_5',
                                            'contact_6',
                                            'contact_7',
                                            'contact_8',
                                            'contact_9',
                                            'Stimulation_Current',
                                            'Stimulation_Frequency',
                                            'Stimulation_PulseWidth',
                                            'Stimulation_Duration',
                                            'Stimulation_ChargeDensity',
                                            'PEI'])

## Get cathodes and anodes 

In [16]:
temp_df = to_df.loc[:, 'contact_1':'contact_9']
#temp_df

In [17]:
cathode = []

for i in range(len(temp_df)):
    if 'null' in  list(temp_df.loc[i,:]):
        cathode.append('null')
    if 'null' not in list(temp_df.loc[i,:]):
        cathode.append([value for value in temp_df.loc[i,:] if '-' in value])
        i+=1
    

In [18]:
anode = []

for i in range(len(temp_df)):
    if 'null' in  list(temp_df.loc[i,:]):
        anode.append('null')
    if 'null' not in list(temp_df.loc[i,:]):
        anode.append([value for value in temp_df.loc[i,:] if '+' in value])
        i+=1

In [19]:
to_df['Stimulation_Cathode'] = cathode
to_df['Stimulation_Anode'] = anode

## Fix timestamps

In [20]:
#Convert timestamps to datetime object
timestamps_list = list(to_df['StartTime'])
timestamps_datetime =  []

for i in range(len(timestamps_list)):
    timestamps_datetime.append(datetime.strptime(timestamps_list[i], '%b %d, %Y %H:%M:%S (US/Pacific)'))
    i+=1

In [21]:
#Replace the old timestamps with the new datetime objects
to_df['StartTime']=timestamps_datetime

In [22]:
#Sort rows by timestamp
to_df = to_df.sort_values(by='StartTime')
to_df = to_df.reset_index(drop=True)
to_df

,StartTime,TherapyLimitPerDay,TherapyLimitResetTime,CaptureWindow,DetectionStatus,StimulationStatus,InversionLogic,PatternALogic,PatternBLogic,DetectorA1_BandpassStatus,...,contact_8,contact_9,Stimulation_Current,Stimulation_Frequency,Stimulation_PulseWidth,Stimulation_Duration,Stimulation_ChargeDensity,PEI,Stimulation_Cathode,Stimulation_Anode
0,2023-07-15 09:46:33,500,06:30,240 secs,Enabled,Enabled,Not Inverted,On (1 OR 2),Off (1 OR 2),On,...,RSGC4 0,Can +,1.0 mA,142.9 Hz,160 µs,3000 ms,1.0 µC/cm²,40,"[RSGC1 -, RSGC2 -]",[Can +]


## Add EndTime

In [23]:
StartTime_col = to_df.loc[:, 'StartTime']
EndTime_col = StartTime_col.drop([0])
EndTime_col = EndTime_col.reset_index(drop=True)

In [24]:
to_df['EndTime']=EndTime_col
#to_df

# Select final table

## Current order

## Suggested order (all data)

In [25]:
final_df = to_df.loc[:, ['StartTime', 'EndTime', 'DetectionStatus', 'StimulationStatus',
                         'TherapyLimitPerDay', 'TherapyLimitResetTime', 'CaptureWindow','InversionLogic',
                         'PatternALogic', 'PatternBLogic', 'DetectorA1_BandpassStatus',
                         'DetectorA1_Channels', 'DetectorA1_MinFreq', 'DetectorA1_MaxFreq',
                         'DetectorA1_MinAmp_ClinMode', 'DetectorA1_DetectAnalysisWindowSize',
                         'DetectorA1_BandpassThreshold', 'DetectorA1_DetectionPersistence',
                         'DetectorA1_MinFreqWindow', 'DetectorA1_MinFreqCountCriteria',
                         'DetectorA1_Hysteresis', 'DetectorA1_MinAmp_TechMode', 'DetectorA1_MinWidth',
                         'DetectorA1_MinDuration',
                         'DetectorA2_BandpassStatus', 'DetectorA2_Channels', 'DetectorA2_MinFreq',
                         'DetectorA2_MaxFreq', 'DetectorA2_MinAmp_ClinMode',
                         'DetectorA2_DetectAnalysisWindowSize', 'DetectorA2_BandpassThreshold',
                         'DetectorA2_DetectionPersistence', 'DetectorA2_MinFreqWindow',
                         'DetectorA2_MinFreqCountCriteria', 'DetectorA2_Hysteresis',
                         'DetectorA2_MinAmp_TechMode', 'DetectorA2_MinWidth', 'DetectorA2_MinDuration',
                         'DetectorB1_BandpassStatus', 'DetectorB1_Channels',
                         'DetectorB1_MinFreq', 'DetectorB1_MaxFreq', 'DetectorB1_MinAmp_ClinMode',
                         'DetectorB1_DetectAnalysisWindowSize', 'DetectorB1_BandpassThreshold',
                         'DetectorB1_DetectionPersistence', 'DetectorB1_MinFreqWindow',
                         'DetectorB1_MinFreqCountCriteria', 'DetectorB1_Hysteresis',
                         'DetectorB1_MinAmp_TechMode', 'DetectorB1_MinWidth', 'DetectorB1_MinDuration',
                         'DetectorB2_BandpassStatus', 'DetectorB2_Channels', 'DetectorB2_MinFreq',
                         'DetectorB2_MaxFreq', 'DetectorB2_MinAmp_ClinMode',
                         'DetectorB2_DetectAnalysisWindowSize', 'DetectorB2_BandpassThreshold',
                         'DetectorB2_DetectionPersistence', 'DetectorB2_MinFreqWindow',
                         'DetectorB2_MinFreqCountCriteria', 'DetectorB2_Hysteresis',
                         'DetectorB2_MinAmp_TechMode', 'DetectorB2_MinWidth', 'DetectorB2_MinDuration',
                         'Stimulation_Cathode', 'Stimulation_Anode', 'Stimulation_Current',
                         'Stimulation_Frequency', 'Stimulation_PulseWidth', 'Stimulation_Duration',
                         'Stimulation_ChargeDensity', 'PEI']]

In [26]:
final_df['CaptureWindow'] = final_df['CaptureWindow'].str.replace('secs','')

final_df['DetectorA1_MinFreq'] = final_df['DetectorA1_MinFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorA1_MinFreq'] = final_df['DetectorA1_MinFreq'].str.replace('Hz Spiking','')
final_df['DetectorA1_MaxFreq'] = final_df['DetectorA1_MaxFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorA1_MinAmp_ClinMode'] = final_df['DetectorA1_MinAmp_ClinMode'].str.replace('%','')
final_df['DetectorA1_DetectAnalysisWindowSize'] = final_df['DetectorA1_DetectAnalysisWindowSize'].str.replace('ms','')
final_df['DetectorA1_DetectionPersistence'] = final_df['DetectorA1_DetectionPersistence'].str.replace('secs','')
final_df['DetectorA1_MinFreqWindow'] = final_df['DetectorA1_MinFreqWindow'].str.replace('ms','')
final_df['DetectorA1_MinWidth'] = final_df['DetectorA1_MinWidth'].str.replace('ms','')
final_df['DetectorA1_MinDuration'] = final_df['DetectorA1_MinDuration'].str.replace('secs','')

final_df['DetectorB1_MinFreq'] = final_df['DetectorB1_MinFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorB1_MinFreq'] = final_df['DetectorB1_MinFreq'].str.replace('Hz Spiking','')
final_df['DetectorB1_MaxFreq'] = final_df['DetectorB1_MaxFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorB1_MinAmp_ClinMode'] = final_df['DetectorB1_MinAmp_ClinMode'].str.replace('%','')
final_df['DetectorB1_DetectAnalysisWindowSize'] = final_df['DetectorB1_DetectAnalysisWindowSize'].str.replace('ms','')
final_df['DetectorB1_DetectionPersistence'] = final_df['DetectorB1_DetectionPersistence'].str.replace('secs','')
final_df['DetectorB1_MinFreqWindow'] = final_df['DetectorB1_MinFreqWindow'].str.replace('ms','')
final_df['DetectorB1_MinWidth'] = final_df['DetectorB1_MinWidth'].str.replace('ms','')
final_df['DetectorB1_MinDuration'] = final_df['DetectorB1_MinDuration'].str.replace('secs','')

final_df['DetectorA2_MinFreq'] = final_df['DetectorA2_MinFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorA2_MinFreq'] = final_df['DetectorA2_MinFreq'].str.replace('Hz Spiking','')
final_df['DetectorA2_MaxFreq'] = final_df['DetectorA2_MaxFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorA2_MinAmp_ClinMode'] = final_df['DetectorA2_MinAmp_ClinMode'].str.replace('%','')
final_df['DetectorA2_DetectAnalysisWindowSize'] = final_df['DetectorA2_DetectAnalysisWindowSize'].str.replace('ms','')
final_df['DetectorA2_DetectionPersistence'] = final_df['DetectorA2_DetectionPersistence'].str.replace('secs','')
final_df['DetectorA2_MinFreqWindow'] = final_df['DetectorA2_MinFreqWindow'].str.replace('ms','')
final_df['DetectorA2_MinWidth'] = final_df['DetectorA2_MinWidth'].str.replace('ms','')
final_df['DetectorA2_MinDuration'] = final_df['DetectorA2_MinDuration'].str.replace('secs','')

final_df['DetectorB2_MinFreq'] = final_df['DetectorB2_MinFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorB2_MinFreq'] = final_df['DetectorB2_MinFreq'].str.replace('Hz Spiking','')
final_df['DetectorB2_MaxFreq'] = final_df['DetectorB2_MaxFreq'].str.replace('Hz Sinusoid','')
final_df['DetectorB2_MinAmp_ClinMode'] = final_df['DetectorB2_MinAmp_ClinMode'].str.replace('%','')
final_df['DetectorB2_DetectAnalysisWindowSize'] = final_df['DetectorB2_DetectAnalysisWindowSize'].str.replace('ms','')
final_df['DetectorB2_DetectionPersistence'] = final_df['DetectorB2_DetectionPersistence'].str.replace('secs','')
final_df['DetectorB2_MinFreqWindow'] = final_df['DetectorB2_MinFreqWindow'].str.replace('ms','')
final_df['DetectorB2_MinWidth'] = final_df['DetectorB2_MinWidth'].str.replace('ms','')
final_df['DetectorB2_MinDuration'] = final_df['DetectorB2_MinDuration'].str.replace('secs','')

final_df['Stimulation_Current'] = final_df['Stimulation_Current'].str.replace('mA','')
final_df['Stimulation_Frequency'] = final_df['Stimulation_Frequency'].str.replace('Hz','')
final_df['Stimulation_PulseWidth'] = final_df['Stimulation_PulseWidth'].str.replace('µs','')
final_df['Stimulation_Duration'] = final_df['Stimulation_Duration'].str.replace('ms','')
final_df['Stimulation_ChargeDensity'] = final_df['Stimulation_ChargeDensity'].str.replace('µC/cm²','')

final_df['PEI'] = final_df['PEI'].str.replace('s','')
final_df['PEI'] = final_df['PEI'].str.replace('min','')

final_df

,StartTime,EndTime,DetectionStatus,StimulationStatus,TherapyLimitPerDay,TherapyLimitResetTime,CaptureWindow,InversionLogic,PatternALogic,PatternBLogic,...,DetectorB2_MinWidth,DetectorB2_MinDuration,Stimulation_Cathode,Stimulation_Anode,Stimulation_Current,Stimulation_Frequency,Stimulation_PulseWidth,Stimulation_Duration,Stimulation_ChargeDensity,PEI
0,2023-07-15 09:46:33,NaT,Enabled,Enabled,500,06:30,240,Not Inverted,On (1 OR 2),Off (1 OR 2),...,null,null,"[RSGC1 -, RSGC2 -]",[Can +],1.0,142.9,160,3000,1.0,40


# Save csv

In [27]:
final_df.to_csv('/home/dastudillo/PDMS/programming_reports/PR04/parsed/PR04_programming_parsed_last.csv')